# Stochastic Minibatch Gradient Descent for Linear Models


In [1]:
# Importing libraries
import numpy as np
import time
import torch
import torch.nn as nn


## Defining functions


In [2]:
# Define the polynomial_fun() function
def polynomial_fun(w, x):
    """
    Calculates the value of a polynomial function at a given point.

    Parameters:
    w (torch.Tensor): Coefficients of the polynomial function.
    x (torch.Tensor): The point(s) at which to evaluate the polynomial function.

    Returns:
    torch.Tensor: The value(s) of the polynomial function at the given point(s).
    """

    # Check if w and x are tensors
    if not isinstance(w, torch.Tensor):
        raise TypeError("w must be a tensor")
    if not isinstance(x, torch.Tensor):
        raise TypeError("x must be a tensor")

    M = w.shape[0]
    terms = [w[i] * torch.pow(x, i) for i in range(M)]
    y = torch.stack(terms, dim=0).sum(dim=0)
    return y


In [3]:
def fit_polynomial_ls(x, t, M):
    """
    Fits a polynomial function to a set of data points using least squares.

    Parameters:
    x (array or list): The x-coordinates of the data points.
    t (array or list): The target values of the data points.
    M (int): The degree of the polynomial to fit to the data.

    Returns:
    array: The coefficients of the polynomial function that best fits the data.
    """
    # Convert x and t to tensors if they are lists
    if isinstance(x, list):
        x = torch.tensor(x)
    if isinstance(t, list):
        t = torch.tensor(t)

    # Make sure the inputs are valid
    ## Type
    if not isinstance(x, torch.Tensor):
        raise TypeError("x must be a tensor or list")
    if not isinstance(t, torch.Tensor):
        raise TypeError("t must be a tensor or list")
    if not isinstance(M, int):
        raise TypeError("M must be an integer")

    # Fit the polynomial function to the data
    A = torch.vander(x, M + 1)
    w = torch.linalg.lstsq(A, t.unsqueeze(1)).solution.squeeze(1)
    return w


In [23]:
def fit_polynomial_sgd(x, t, M, learning_rate, minibatch_size, period=100):
    """
    Fits a polynomial function to a set of data points using stochastic gradient descent.

    Parameters:
    x (torch.Tensor): The x-coordinates of the data points.
    t (torch.Tensor): The target values of the data points.
    M (int): The degree of the polynomial to fit to the data.
    learning_rate (float): The learning rate to use for the SGD algorithm.
    minibatch_size (int): The number of data points to use in each minibatch.
    period (int): The number of epochs to run the SGD algorithm.

    Returns:
    torch.Tensor: The coefficients of the polynomial function that best fits the data.

    Prints:
    float: The loss value at each 10% epoch.
    """
    # Make sure the inputs are valid
    if not isinstance(x, torch.Tensor):
        raise TypeError("x must be a tensor")
    if not isinstance(t, torch.Tensor):
        raise TypeError("t must be a tensor")
    if not isinstance(M, int):
        raise TypeError("M must be an integer")
    if not isinstance(learning_rate, float):
        raise TypeError("learning_rate must be a float")
    if not isinstance(minibatch_size, int):
        raise TypeError("minibatch_size must be an integer")

    # Fit the polynomial function to the data
    w = torch.randn(M + 1, requires_grad=True)
    for epoch in range(period):
        indices = torch.randperm(x.shape[0])[:minibatch_size]
        x_batch = x[indices]
        t_batch = t[indices]
        A = torch.vander(x_batch, M + 1)
        y = torch.mv(A, w)
        loss = torch.mean((y - t_batch) ** 2)
        loss.backward()
        with torch.no_grad():
            w -= learning_rate * w.grad
            w.grad.zero_()

        if epoch % (period // 10) == 0:
            print(f"Epoch {epoch}: Loss = {loss.item()}")

    return w


## Questions


### a. Generating the training and test data


In [24]:
import torch

# Define the input data
M = 2
w_T = torch.tensor([1, 2, 3])
w = w_T.T


# Generate training set
torch.manual_seed(0)
x_train = torch.rand(20) * 40 - 20  # random values between -20 and 20
y_train_gt = polynomial_fun(w, x_train)  # ground truth
t_train = y_train_gt + torch.randn(20) * 0.5

# Generate test set
x_test = torch.rand(10) * 40 - 20
y_test_gt = polynomial_fun(w, x_test)  # ground truth
t_test = y_test_gt + torch.randn(10) * 0.5


In [25]:
print(f"x_train: {x_train}")
print(f"y_train_gt: {y_train_gt}")
print(f"t_train: {t_train}")

print("------")


print(f"x_test: {x_test}")
print(f"y_test_gt: {y_test_gt}")
print(f"t_test: {t_test}")


x_train: tensor([ -0.1497,  10.7289, -16.4609, -14.7188,  -7.7031,   5.3631,  -0.3963,
         15.8578,  -1.7749,   5.2923,  -6.0443,  -3.9313, -19.1070, -13.2456,
         -8.2445,   0.7409,   7.9067,  12.0005, -13.5588,  -8.7093])
y_train_gt: tensor([7.6779e-01, 3.6778e+02, 7.8096e+02, 6.2149e+02, 1.6361e+02, 9.8016e+01,
        6.7855e-01, 7.8712e+02, 6.9008e+00, 9.5608e+01, 9.8511e+01, 3.9503e+01,
        1.0580e+03, 5.0085e+02, 1.8842e+02, 4.1284e+00, 2.0436e+02, 4.5703e+02,
        5.2541e+02, 2.1113e+02])
t_train: tensor([1.0672e+00, 3.6701e+02, 7.8079e+02, 6.2242e+02, 1.6350e+02, 9.7645e+01,
        9.5991e-01, 7.8725e+02, 6.8139e+00, 9.5269e+01, 9.8980e+01, 3.9747e+01,
        1.0586e+03, 5.0089e+02, 1.8782e+02, 4.1260e+00, 2.0410e+02, 4.5688e+02,
        5.2462e+02, 2.1199e+02])
------
x_test: tensor([  3.7269, -15.5061, -13.8617, -10.3317,   9.0495,   8.0432, -11.8470,
          6.0421,  10.9794,  -2.5243])
y_test_gt: tensor([ 50.1227, 691.3062, 549.7187, 300.5670, 264.7772

### b. Fitting the training set - LeastSquare


In [26]:
# Fit polynomial function using fit_polynomial_lstsq

## Setting
M_values = [2, 3, 4]

## Initalising
w_hat_values = []
y_train_values = []
y_test_values = []
lstsq_fit_times = []
lstsq_train_times = []
lstsq_test_times = []

## Looping
print(f"LEAST SQUARE EVALUATION")
for M in M_values:
    print(f"M={M}")

    # Compute the optimum weight vector
    start_time = time.time()
    w_hat = fit_polynomial_ls(x_train, t_train, M)
    end_time = time.time()
    lstsq_fit_times.append(end_time - start_time)
    w_hat_values.append(w_hat)

    # Compute predicted target values for training set
    start_time = time.time()
    y_train = polynomial_fun(w_hat, x_train)
    end_time = time.time()
    lstsq_train_times.append(end_time - start_time)
    y_train_values.append(y_train)

    # Compute predicted target values for test set
    start_time = time.time()
    y_test = polynomial_fun(w_hat, x_test)
    end_time = time.time()
    lstsq_test_times.append(end_time - start_time)
    y_test_values.append(y_test)

    #! TODO: Meansquare
    # Reporting a

    # Calculate the mean and standard deviation of the difference between the observed training data and the true polynomial curve
    print(f"a) Difference between observed training data and true polynomial curve")
    diff_train = t_train - polynomial_fun(w, x_train)
    mean_diff_train = diff_train.mean()
    std_diff_train = diff_train.std()
    print(
        f"Mean difference between observed training data and true polynomial curve: {mean_diff_train}"
    )
    print(
        f"Standard deviation of difference between observed training data and true polynomial curve: {std_diff_train}"
    )

    # Reporting b
    # Calculate the mean and standard deviation of the difference between the LS-predicted values and the true polynomial curve
    print(f"b) Difference between LS-predicted values and true polynomial curve")
    diff_ls_predicted = y_train - polynomial_fun(w, x_train)
    mean_diff_ls_predicted = diff_ls_predicted.mean()
    std_diff_ls_predicted = diff_ls_predicted.std()
    print(
        f"Mean difference between LS-predicted values and true polynomial curve: {mean_diff_ls_predicted}"
    )
    print(
        f"Standard deviation of difference between LS-predicted values and true polynomial curve: {std_diff_ls_predicted}"
    )

    print("---------------------------------------------------")


LEAST SQUARE EVALUATION
M=2
a) Difference between observed training data and true polynomial curve
Mean difference between observed training data and true polynomial curve: 0.009434586390852928
Standard deviation of difference between observed training data and true polynomial curve: 0.48098304867744446
b) Difference between LS-predicted values and true polynomial curve
Mean difference between LS-predicted values and true polynomial curve: -212.5789337158203
Standard deviation of difference between LS-predicted values and true polynomial curve: 217.66619873046875
---------------------------------------------------
M=3
a) Difference between observed training data and true polynomial curve
Mean difference between observed training data and true polynomial curve: 0.009434586390852928
Standard deviation of difference between observed training data and true polynomial curve: 0.48098304867744446
b) Difference between LS-predicted values and true polynomial curve
Mean difference between LS-pr

### c. Fitting the training set - SGD


In [27]:
import torch
import time

# Assuming you have x_train, t_train, x_test, and t_test defined

# Hyperparameters
M_values = [2, 3, 4]
learning_rate = 1e-8
minibatch_size = 2
epochs = 100

# Fit the true polynomial function using least squares
w = fit_polynomial_ls(x_train, t_train, M_values[0])

# Fit polynomial function using fit_polynomial_sgd
sgd_w_hat_values = []
sgd_y_train_values = []
sgd_y_test_values = []
sgd_fit_times = []
sgd_train_times = []
sgd_test_times = []

print("SGD EVALUATION")
for M in M_values:
    print(f"M={M}")

    # Fit the polynomial function using SGD
    # => sgd_w_hat
    start_time = time.time()
    sgd_w_hat = fit_polynomial_sgd(
        x_train, t_train, M, learning_rate, minibatch_size, epochs
    )
    end_time = time.time()
    sgd_fit_times.append(end_time - start_time)
    sgd_w_hat_values.append(sgd_w_hat)

    # Compute predicted target values for training set using SGD
    # => sgd_y_train
    start_time = time.time()
    sgd_y_train = polynomial_fun(sgd_w_hat, x_train)
    end_time = time.time()
    sgd_train_times.append(end_time - start_time)
    sgd_y_train_values.append(sgd_y_train)

    # Compute predicted target values for test set using SGD
    # => sgd_y_test
    start_time = time.time()
    sgd_y_test = polynomial_fun(sgd_w_hat, x_test)
    end_time = time.time()
    sgd_test_times.append(end_time - start_time)
    sgd_y_test_values.append(sgd_y_test)

    # Difference between the SGD-predicted values and the true polynomial curve
    print("Difference between observed training data and true polynomial curve")
    diff_sgd_predicted_train = sgd_y_train - polynomial_fun(w, x_train)
    mean_diff_sgd_predicted_train = diff_sgd_predicted_train.mean()
    std_diff_sgd_predicted_train = diff_sgd_predicted_train.std()
    print(
        f"Mean difference between SGD-predicted values and true polynomial curve for M={M}: {mean_diff_sgd_predicted_train}"
    )
    print(
        f"Standard deviation of difference between SGD-predicted values and true polynomial curve for M={M}: {std_diff_sgd_predicted_train}"
    )

    # Difference between the SGD-predicted values and the true polynomial curve for the test set
    print(
        "Difference between the SGD-predicted values and the true polynomial curve for the test set"
    )
    diff_sgd_predicted_test = sgd_y_test - polynomial_fun(w, x_test)
    mean_diff_sgd_predicted_test = diff_sgd_predicted_test.mean()
    std_diff_sgd_predicted_test = diff_sgd_predicted_test.std()
    print(
        f"Mean difference between SGD-predicted values and true polynomial curve for M={M} (test set): {mean_diff_sgd_predicted_test}"
    )
    print(
        f"Standard deviation of difference between SGD-predicted values and true polynomial curve for M={M} (test set): {std_diff_sgd_predicted_test}"
    )

    print("---------------------------------------------------")


SGD EVALUATION
M=2
Epoch 0: Loss = 483663.90625
Epoch 10: Loss = 335765.40625
Epoch 20: Loss = 304490.53125
Epoch 30: Loss = 886467.375
Epoch 40: Loss = 9856.11328125
Epoch 50: Loss = 191692.8125
Epoch 60: Loss = 459068.03125
Epoch 70: Loss = 115772.5625
Epoch 80: Loss = 595245.5
Epoch 90: Loss = 1449.019287109375
Difference between observed training data and true polynomial curve
Mean difference between SGD-predicted values and true polynomial curve for M=2: -306.2291564941406
Standard deviation of difference between SGD-predicted values and true polynomial curve for M=2: 306.3013610839844
Difference between the SGD-predicted values and the true polynomial curve for the test set
Mean difference between SGD-predicted values and true polynomial curve for M=2 (test set): -293.73138427734375
Standard deviation of difference between SGD-predicted values and true polynomial curve for M=2 (test set): 214.8321990966797
---------------------------------------------------
M=3
Epoch 0: Loss = 10

In [29]:
# Compare the speed of the two methods
print(f"SPEED COMPARISON")
print(f"Least squares")
print(f"Time spent in fitting using least squares: {np.sum(lstsq_fit_times)} seconds")
print(
    f"Time spent in training using least squares: {np.sum(lstsq_train_times)} seconds"
)
print("-----------------")
print("SGD times")
print(f"Time spent in fitting using SGD: {np.sum(sgd_fit_times)} seconds")
print(f"Time spent in training using SGD: {np.sum(sgd_train_times)} seconds")


SPEED COMPARISON
Least squares
Time spent in fitting using least squares: 0.0005087852478027344 seconds
Time spent in training using least squares: 0.0002117156982421875 seconds
-----------------
SGD times
Time spent in fitting using SGD: 0.023846149444580078 seconds
Time spent in training using SGD: 0.0001971721649169922 seconds


## Optimise with M


In [30]:
import torch


def fit_polynomial_sgd_M(x, t, learning_rate, minibatch_size, epochs, M_init=2):
    """
    Fits a polynomial function to a set of data points using stochastic gradient descent.

    Parameters:
    x (torch.Tensor): The x-coordinates of the data points.
    t (torch.Tensor): The target values of the data points.
    learning_rate (float): The learning rate to use for the SGD algorithm.
    minibatch_size (int): The number of data points to use in each minibatch.
    epochs (int): The number of epochs to run the SGD algorithm.
    M_init (int): The initial value of the polynomial degree M.

    Returns:
    torch.Tensor: The coefficients of the polynomial function that best fits the data.
    int: The optimized value of the polynomial degree M.

    Prints:
    float: The loss value at each 10% epoch.
    """
    # Make sure the inputs are valid
    if not isinstance(x, torch.Tensor):
        raise TypeError("x must be a tensor")
    if not isinstance(t, torch.Tensor):
        raise TypeError("t must be a tensor")
    if not isinstance(learning_rate, float):
        raise TypeError("learning_rate must be a float")
    if not isinstance(minibatch_size, int):
        raise TypeError("minibatch_size must be an integer")
    if not isinstance(epochs, int):
        raise TypeError("epochs must be an integer")
    if not isinstance(M_init, int):
        raise TypeError("M_init must be an integer")

    # Fit the polynomial function to the data
    w = torch.randn(M_init + 1, requires_grad=True)
    M = torch.tensor(float(M_init), requires_grad=True)  # Convert M_init to float
    optimizer = torch.optim.SGD([w, M], lr=learning_rate)

    for epoch in range(epochs):
        indices = torch.randperm(x.shape[0])[:minibatch_size]
        x_batch = x[indices]
        t_batch = t[indices]
        A = torch.vander(x_batch, int(M.item()) + 1)
        y = torch.mv(A, w)
        loss = torch.mean((y - t_batch) ** 2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % (epochs // 10) == 0:
            print(f"Epoch {epoch}: Loss = {loss.item()}")

    M_optimized = int(M.item())
    return w, M_optimized


In [31]:
import torch

# Assuming you have x_train, t_train, x_test, and t_test defined

# Hyperparameters
learning_rate = 0.01
minibatch_size = 2
epochs = 1000

# Fit the true polynomial function using least squares
w_true = fit_polynomial_ls(x_train, t_train, M_values[0])

# Fit the polynomial function using fit_polynomial_sgd
sgd_w_hat, sgd_M_optimized = fit_polynomial_sgd_M(
    x_train, t_train, learning_rate, minibatch_size, epochs
)

# Compute predicted target values for training set using SGD
sgd_y_train = polynomial_fun(sgd_w_hat, x_train)

# Compute predicted target values for test set using SGD
sgd_y_test = polynomial_fun(sgd_w_hat, x_test)

# Difference between the SGD-predicted values and the true polynomial curve
print(f"Optimized value of M: {sgd_M_optimized}")

print("Difference between observed training data and true polynomial curve")
diff_sgd_predicted_train = sgd_y_train - polynomial_fun(w_true, x_train)
mean_diff_sgd_predicted_train = diff_sgd_predicted_train.mean()
std_diff_sgd_predicted_train = diff_sgd_predicted_train.std()
print(
    f"Mean difference between SGD-predicted values and true polynomial curve: {mean_diff_sgd_predicted_train}"
)
print(
    f"Standard deviation of difference between SGD-predicted values and true polynomial curve: {std_diff_sgd_predicted_train}"
)

# Difference between the SGD-predicted values and the true polynomial curve for the test set
print(
    "Difference between the SGD-predicted values and the true polynomial curve for the test set"
)
diff_sgd_predicted_test = sgd_y_test - polynomial_fun(w_true, x_test)
mean_diff_sgd_predicted_test = diff_sgd_predicted_test.mean()
std_diff_sgd_predicted_test = diff_sgd_predicted_test.std()
print(
    f"Mean difference between SGD-predicted values and true polynomial curve (test set): {mean_diff_sgd_predicted_test}"
)
print(
    f"Standard deviation of difference between SGD-predicted values and true polynomial curve (test set): {std_diff_sgd_predicted_test}"
)


Epoch 0: Loss = 453697.5
Epoch 100: Loss = nan
Epoch 200: Loss = nan
Epoch 300: Loss = nan
Epoch 400: Loss = nan
Epoch 500: Loss = nan
Epoch 600: Loss = nan
Epoch 700: Loss = nan
Epoch 800: Loss = nan
Epoch 900: Loss = nan
Optimized value of M: 2
Difference between observed training data and true polynomial curve
Mean difference between SGD-predicted values and true polynomial curve: nan
Standard deviation of difference between SGD-predicted values and true polynomial curve: nan
Difference between the SGD-predicted values and the true polynomial curve for the test set
Mean difference between SGD-predicted values and true polynomial curve (test set): nan
Standard deviation of difference between SGD-predicted values and true polynomial curve (test set): nan


In [34]:
import torch

# Assuming you have x_train, t_train, x_test, and t_test defined

# Hyperparameters
learning_rate = 1e-8
minibatch_size = 2
epochs = 100
M_values = [2, 3, 4, 5]  # List of M values to try

# Fit the true polynomial function using least squares
w_true = fit_polynomial_ls(x_train, t_train, M_values[0])

best_M = None
best_w_hat = None
best_train_diff = float("inf")
best_test_diff = float("inf")

for M in M_values:
    print(f"Trying M={M}")

    # Fit the polynomial function using fit_polynomial_sgd
    sgd_w_hat = fit_polynomial_sgd(
        x_train, t_train, M, learning_rate, minibatch_size, epochs
    )

    # Compute predicted target values for training set using SGD
    sgd_y_train = polynomial_fun(sgd_w_hat, x_train)

    # Compute predicted target values for test set using SGD
    sgd_y_test = polynomial_fun(sgd_w_hat, x_test)

    # Difference between the SGD-predicted values and the true polynomial curve
    diff_sgd_predicted_train = sgd_y_train - polynomial_fun(w_true, x_train)
    mean_diff_sgd_predicted_train = diff_sgd_predicted_train.mean().item()

    diff_sgd_predicted_test = sgd_y_test - polynomial_fun(w_true, x_test)
    mean_diff_sgd_predicted_test = diff_sgd_predicted_test.mean().item()

    if (
        mean_diff_sgd_predicted_train < best_train_diff
        and mean_diff_sgd_predicted_test < best_test_diff
    ):
        best_M = M
        best_w_hat = sgd_w_hat
        best_train_diff = mean_diff_sgd_predicted_train
        best_test_diff = mean_diff_sgd_predicted_test

print(f"Optimized value of M: {best_M}")
print(
    f"Mean difference between SGD-predicted values and true polynomial curve (training set): {best_train_diff}"
)
print(
    f"Mean difference between SGD-predicted values and true polynomial curve (test set): {best_test_diff}"
)


Trying M=2
Epoch 0: Loss = 7341.1220703125
Epoch 10: Loss = 3147.911865234375
Epoch 20: Loss = 327439.875
Epoch 30: Loss = 4.4719648361206055
Epoch 40: Loss = 187536.890625
Epoch 50: Loss = 104310.265625
Epoch 60: Loss = 78789.4609375
Epoch 70: Loss = 101940.3125
Epoch 80: Loss = 261221.375
Epoch 90: Loss = 158084.90625
Trying M=3
Epoch 0: Loss = 4210033.0
Epoch 10: Loss = 7880.95849609375
Epoch 20: Loss = 252684.359375
Epoch 30: Loss = 356505.46875
Epoch 40: Loss = 9718.95703125
Epoch 50: Loss = 143256.984375
Epoch 60: Loss = 289911.75
Epoch 70: Loss = 9039.970703125
Epoch 80: Loss = 178830.65625
Epoch 90: Loss = 11050.654296875
Trying M=4
Epoch 0: Loss = 166214.078125
Epoch 10: Loss = 1.007356517738948e+25
Epoch 20: Loss = inf
Epoch 30: Loss = inf
Epoch 40: Loss = nan
Epoch 50: Loss = nan
Epoch 60: Loss = nan
Epoch 70: Loss = nan
Epoch 80: Loss = nan
Epoch 90: Loss = nan
Trying M=5
Epoch 0: Loss = 13773918208.0
Epoch 10: Loss = inf
Epoch 20: Loss = nan
Epoch 30: Loss = nan
Epoch 40: 